In [1]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

import pandas, seaborn, numpy, collections, sefara, varcode, uncertainties, numpy, statsmodels, pickle, scipy
%matplotlib inline
from matplotlib import pyplot
import topiary
import mhctools
import matplotlib_venn
import intervaltree
import statsmodels.robust
import mhcflurry
from varcode.common import memoize
pandas.set_option('display.max_columns', None)
import scipy, sys, sklearn
import sklearn.utils
import matplotlib
#import common as c
import sklearn.tree
import pymc3
from scipy import optimize
import glob


def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')

matplotlib.rc("savefig", dpi=800)
#matplotlib.rc("savefig", dpi=72)
matplotlib.rc('text', usetex=False)
#reload(c)

%matplotlib inline
%config InlineBackend.figure_format = 'png'

import pystan
import pyfaidx



/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [2]:
# Write out mutation counts
mutations = pandas.read_csv("../data/mutations.csv.bz2", converters={'binding_peptides': eval})
mutations = mutations.ix[mutations.source_id.str.startswith("AOCS")]
mutations

KeyboardInterrupt: 

In [ ]:
sources = pandas.read_csv("../data/sources.csv", index_col="source_id")
sources["source_id"] = sources.index
sources = sources.ix[sources.cohort == "AOCS"]
treated_paired_donors = set(d for d in sources.donor
                              if len(sources.ix[sources.donor == d].treated.unique()) > 1 )
treated_paired_donors
sources["treated_paired"] = sources.donor.isin(treated_paired_donors)
sources

In [ ]:
muts_new = mutations.ix[[
    (sources.ix[s].treated) for s in mutations.source_id
]].ix[
    mutations.snv &
    mutations.donor.isin(treated_paired_donors) &
    mutations.unique_to_treated
].groupby(["source_id", "context_mutation_3p5p"]).interbase_start.count().fillna(0).unstack().fillna(0)
muts_new["kind"] = "new"
muts_new

In [ ]:
muts_all = mutations.ix[
    mutations.snv
].groupby(["source_id", "context_mutation_3p5p"]).interbase_start.count().unstack().T.fillna(0)
#muts_all.columns = ["%s__all" % (x.replace("/", "_")) for x in muts_all.columns]
muts_all = muts_all.T
muts_all["kind"] = "all"

print(muts_all.isnull().sum().sum())
muts_all


In [ ]:
# To be loaded into R to use deconstructSigs package
muts_r = pandas.concat([muts_all, muts_new])

muts_r.columns = [x.replace("(", "[").replace(")", "]") for x in muts_r.columns]
muts_r["source_id"] = muts_r.index
muts_r = muts_r[["source_id"] + list(muts_r.columns)[:-1]]
muts_r.to_csv("../data/mutation_contexts_counts.csv", index=False)
muts_r